In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display_html 
from IPython import display
import sys
sys.path.append('C:/Users/User/Desktop/StockInfoHub')
from stock_rs_rate_analysis_package.stock_rs_rate_analysis_utils import *
from Shared_Modules.shared_variables import *
import warnings
import requests
warnings.filterwarnings('ignore')
class bcolors:
    OK = '\033[92m' #GREEN
    WARNING = '\033[93m' #YELLOW
    FAIL = '\033[91m' #RED
    RESET = '\033[0m' #RESET COLOR

Shared module initialized.
['2023-10-09', '2023-10-10', '2024-01-01', '2024-02-06', '2024-02-07', '2024-02-08', '2024-02-09', '2024-02-10', '2024-02-11', '2024-02-12', '2024-02-13', '2024-02-14', '2024-02-28', '2024-04-04', '2024-04-05', '2024-05-01', '2024-06-10', '2024-09-17', '2024-10-10']


In [3]:
industry_df = pd.read_excel(r'C:\Users\User\Desktop\StockInfoHub\others\產業別.xlsx').astype(int).astype(str)
group_df = pd.read_excel(r'C:\Users\User\Desktop\StockInfoHub\others\族群_複製.xlsx').astype(int).astype(str)
concept_df = pd.read_excel(r'C:\Users\User\Desktop\StockInfoHub\others\概念股_複製.xlsx').astype(int).astype(str)
rs = pd.read_excel(r'C:\Users\User\Desktop\StockInfoHub\Stock_Data_Collector\250RS選股\2024-03-22250RS選股.xlsx')['ID'].astype(str)
rschoose_df = pd.read_excel(r'C:\Users\User\Desktop\StockInfoHub\Stock_Data_Collector\全個股條件篩選\2024-03-22選股.xlsx')
# 建立所有股票產業
all = []
for id in rs:
    stock_ind = []
    for col in industry_df.columns.values:
        if id in industry_df[col].values:
            stock_ind.append(col)
            all.append([col, id, 1])
all_df = pd.DataFrame(all, columns=['industry', 'ID', 'count'])
all2 = []
for id in rschoose_df['ID'].astype(str):
    stock_ind = []
    for col in industry_df.columns.values:
        if id in industry_df[col].values:
            stock_ind.append(col)
            all2.append([col, id, 1])
    for col in group_df.columns.values:
        if id in group_df[col].values:
            stock_ind.append(col)
            all2.append([col, id, 1])
    for col in concept_df.columns.values:
        if id in concept_df[col].values:
            stock_ind.append(col)
            all2.append([col, id, 1])
all_df2 = pd.DataFrame(all2, columns=['industry', 'ID', 'count'])

In [ ]:
# 看每周被刪除、新增的產業
now_date = '2024-03-14'
old_date = '2024-03-01'
path = r'C:\Users\User\Desktop\StockInfoHub\Stock_Data_Collector\250RS選股\\'
now_ID = pd.read_excel(path+now_date+'EMA250RS選股.xlsx')
previous_ID = pd.read_excel(path+old_date+'EMA250RS選股.xlsx')
now_ID = now_ID.loc[:, 'ID'].values.astype(str)
previous_ID = previous_ID.loc[:, 'ID'].values.astype(str)
dropstock = np.sort([i for i in previous_ID if i not in now_ID]).astype(str)
addstock = np.sort([i for i in now_ID if i not in previous_ID]).astype(str)
def show_diff(drop_category = None, add_category = None):
    drop_industry_category_df = []
    add_industry_category_df = []
    drop_industry_list = []
    add_industry_list = []
    for id in dropstock:
        stock_ind = []
        for col in industry_df.columns.values:
            if id in industry_df[col].values:
                stock_ind.append(col)
        drop_industry_list.append((id,stock_ind))
    for ind in drop_industry_list:
        for i in ind[1]:
            drop_industry_category_df.append([ind[0],i])

    for id in addstock:
        stock_ind = []
        for col in industry_df.columns.values:
            if id in industry_df[col].values:
                stock_ind.append(col)
        add_industry_list.append((id,stock_ind))
    for ind in add_industry_list:
        for i in ind[1]: 
            add_industry_category_df.append([ind[0], i])

    drop_industry_category_df = pd.DataFrame(drop_industry_category_df,columns=['ID', 'category'])
    if drop_category != None:
        print(drop_category, '刪除股票 : ', drop_industry_category_df.loc[drop_industry_category_df['category']==drop_category, 'ID'].values)
    drop_industry_category_df['drop counts'] = 1
    drop_industry_category_df['add counts'] = 0
    drop_industry_category_df = drop_industry_category_df.groupby('category').sum()
    # print(drop_industry_category_df)
    # display(drop_industry_category_df.groupby('category').sum().sort_values(by='count', ascending=False))
    add_industry_category_df = pd.DataFrame(add_industry_category_df,columns=['ID', 'category'])
    if add_category != None:
        print(add_category, '新增股票 : ',add_industry_category_df.loc[add_industry_category_df['category']==add_category, 'ID'].values)
    add_industry_category_df['drop counts'] = 0
    add_industry_category_df['add counts'] = 1
    add_industry_category_df = add_industry_category_df.groupby('category').sum()
    # print(add_industry_category_df)
    industry_category_sum_df = pd.concat([add_industry_category_df, drop_industry_category_df], axis=0)
    industry_category_sum_df = industry_category_sum_df.groupby(by='category').sum()
    industry_category_sum_df['total counts'] = industry_category_sum_df['add counts']-industry_category_sum_df['drop counts']
    n = 20
    df1 = (industry_category_sum_df.sort_values('total counts', ascending=True).iloc[:n])
    df2 = (industry_category_sum_df.sort_values('total counts', ascending=False).iloc[:n])

    df1_styler = df1.style.set_table_attributes("style='display:inline'").set_caption('Get Weaker')
    df2_styler = df2.style.set_table_attributes("style='display:inline'").set_caption('Get Stronger')
    # df2_t_styler = df2.T.style.set_table_attributes("style='display:inline'").set_caption('df2_t')

    display_html(df1_styler._repr_html_()+df2_styler._repr_html_(), raw=True)
    
show_diff('設備儀器廠商', '設備儀器廠商')

In [36]:
#看個股產業
individual = '6715 4770 3050 1603 6442'.split(' ')
to_watch = individual
##########################
alist = []
industry_category_df = []
show = True


CF_df = rschoose_df
CF_df['CF'] = [round(100-(i/14), 1) for i in CF_df.index.values]

for id in to_watch:
    stock_ind = []
    stock_ind.append(rschoose_df.loc[rschoose_df['ID'].astype(str) == id, 'ES250rate'].values[0])
    stock_ind.append(CF_df.loc[CF_df['ID'].astype(str) == id, 'CF'].values[0])
    for col in industry_df.columns.values:
        if id in industry_df[col].values:
            stock_ind.append(col)
    for col in group_df.columns.values:
        if id in group_df[col].values:
            stock_ind.append(col)
    for col in concept_df.columns.values:
        if id in concept_df[col].values:
            stock_ind.append(col)
    alist.append((id,stock_ind))

for ind in alist:
    if show:
        print(ind[0], f'| RS({ind[1][0]}) | CF({ind[1][1]})', '-->', end=' ')
    for i in ind[1][2:]:
        if show:
            print(i, end=' ')
        industry_category_df.append([ind[0], i])
    print('')
industry_category_df = pd.DataFrame(industry_category_df,columns=['ID', 'category'])
industry_category_df['count'] = 1
industry_category_df = industry_category_df.groupby('category').sum()
count_df = industry_category_df.sort_values(by='count', ascending=False)
display.display(count_df)

6715  | RS(83.5) | CF(54.93) | 電子零件元件 傳輸介面 電子上游_連接元件 連接器 TypeC 
4770  | RS(81.9) | CF(43.71) | 化學工業 傳產其他 電子上游_PCB_材料設備 
3050  | RS(86.7) | CF(34.86) | 光碟片 太陽能 旅館、餐飲 休閒娛樂 電子下游_光碟片 太陽能 
1603  | RS(72.9) | CF(8.71) | 電線電纜 建材 傳產_電線電纜 
6442  | RS(97.6) | CF(52.21) | 電子零件元件 光通訊 電子中游_通訊設備 連接器 


,count
category,
電子零件元件,2
連接器,2
太陽能,2
TypeC,1
旅館、餐飲,1
電子中游_通訊設備,1
電子下游_光碟片,1
電子上游_連接元件,1
電子上游_PCB_材料設備,1


In [3]:
d = '04-03'
industry_df = pd.read_excel(r'C:\Users\User\Desktop\StockInfoHub\others\產業別.xlsx').astype(int).astype(str)
group_df = pd.read_excel(r'C:\Users\User\Desktop\StockInfoHub\others\族群_複製.xlsx').astype(int).astype(str)
concept_df = pd.read_excel(r'C:\Users\User\Desktop\StockInfoHub\others\概念股_複製.xlsx').astype(int).astype(str)
rs = pd.read_excel(fr'C:\Users\User\Desktop\StockInfoHub\Stock_Data_Collector\250RS選股\2024-{d}250RS選股.xlsx')['ID'].astype(str)
rschoose_df = pd.read_excel(fr'C:\Users\User\Desktop\StockInfoHub\Stock_Data_Collector\全個股條件篩選\2024-{d}選股.xlsx')

def get_tradingview_format():
    
    
    # ============上市股票df============
    url = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=1&issuetype=1&industry_code=&Page=1&chklike=Y"
    response = requests.get(url, timeout=5)
    print(response.status_code)
    listed = pd.read_html(response.text)[0]
    listed.columns = listed.iloc[0,:]
    listed = listed[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
    listed = listed.iloc[1:]

    # ============上櫃股票df============
    urlTWO = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=2&issuetype=&industry_code=&Page=1&chklike=Y"
    response = requests.get(urlTWO, timeout=5)
    print(response.status_code)
    listedTWO = pd.read_html(response.text)[0]
    listedTWO.columns = listedTWO.iloc[0,:]
    listedTWO = listedTWO.loc[listedTWO['有價證券別'] == '股票']
    listedTWO = listedTWO[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]

    # ============上市股票代號+.TW============
    stock_1 = listed["有價證券代號"]
    stock_num = stock_1.apply(lambda x: str(x) + ".TW")
    stock_num.loc[len(stock_num)+1] = '0050.TW'
    stock_num.loc[len(stock_num)+1] = '^TWII'
    # print(stock_num)

    # ============上櫃股票代號+.TWO============
    stock_2 = listedTWO["有價證券代號"]
    stock_num2 = stock_2.apply(lambda x: str(x) + ".TWO")
    # print(stock_num2)

    # ============concate全部股票代號============
    stock_num = pd.concat([stock_num, stock_num2], ignore_index=True)
    # print(stock_num)
    allstock_info = pd.concat([listed, listedTWO], ignore_index=True)
    allstock_info.columns = ["ID","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]
    allstock_info.set_index('ID', inplace = True)
    # print(allstock_info)
    return allstock_info
allstock_info = get_tradingview_format()
# 建立所有股票產業
all = []
for id in rs:
    stock_ind = []
    for col in industry_df.columns.values:
        if id in industry_df[col].values:
            stock_ind.append(col)
            all.append([col, id, 1])
all_df = pd.DataFrame(all, columns=['industry', 'ID', 'count'])
all2 = []
for id in rschoose_df['ID'].astype(str):
    stock_ind = []
    for col in industry_df.columns.values:
        if id in industry_df[col].values:
            stock_ind.append(col)
            all2.append([col, id, 1])
    for col in group_df.columns.values:
        if id in group_df[col].values:
            stock_ind.append(col)
            all2.append([col, id, 1])
    for col in concept_df.columns.values:
        if id in concept_df[col].values:
            stock_ind.append(col)
            all2.append([col, id, 1])
all_df2 = pd.DataFrame(all2, columns=['industry', 'ID', 'count'])

200
200


In [41]:
from IPython.display import clear_output
from time import sleep as delay
    # 看特定產業股票的產業

show = True
show_all = False
save_to_txt = True
if show_all:
    CF_threshold = 1800
    ES_threshold = 1800
else:
    CF_threshold = 800
    ES_threshold = 500
# variable == intersection or union
mode = 'union'
industry_name_list = '連接器'.split('。')
for industry_name in industry_name_list:
    alist = []
    industry_category_df = []
    clear_output()
    sort_stocks = rschoose_df.sort_values(by='busness volume(億)', ascending=False).iloc[:CF_threshold]
    sort_stocks['CF'] = [round(100-(i/14), 1) for i in sort_stocks.index.values]
    # ES250前20%
    sort_stocks_ES = rschoose_df.sort_values(by='ES250rate', ascending=False).iloc[:ES_threshold]
    print(f'ES top {round(ES_threshold/14, 2)}% : {sort_stocks_ES.shape} | CF top {round(CF_threshold/14, 2)}% : {sort_stocks.shape}')
    # 合併共有股票
    sort_stocks = sort_stocks.loc[sort_stocks['ID'].isin(sort_stocks_ES['ID'])]
    print(f'CF top {round(CF_threshold/14, 2)}% & ES top {round(ES_threshold/14, 2)}% : {sort_stocks.shape}')
    # 特定產業
    industry_name = industry_name.split(' ')
    if mode == 'intersection':
        for i, industry in enumerate(industry_name):
            industry_ID_list = all_df2.loc[all_df2['industry'].isin([industry])]['ID'].values
            print(f'{industry} : {industry_ID_list.shape}')
            if i == 0:
                industry_stock = industry_ID_list
            else:
                industry_stock = np.intersect1d(industry_stock, industry_ID_list)      
    else:
        industry_stock = all_df2.loc[all_df2['industry'].isin(industry_name)]['ID'].values
        print(f'{industry_name} : {industry_stock.shape}')

    print(f'{industry_name} : {industry_stock.shape}')
    sort_stocks_id = sort_stocks.loc[sort_stocks['ID'].astype(str).isin(industry_stock)]['ID'].astype(str).values
    # sort by stock ID
    sort_stocks_id = sort_stocks_id[np.argsort(sort_stocks_id)]
    print(f'{industry_name} after fillter : {sort_stocks.shape}')
    
    # 全部的產業股票
    for id in sort_stocks_id:
        stock_ind = []
        stock_ind.append(rschoose_df.loc[rschoose_df['ID'].astype(str) == id, 'ES250rate'].values[0])
        stock_ind.append(sort_stocks.loc[sort_stocks['ID'].astype(str) == id, 'CF'].values[0])
        for col in industry_df.columns.values:
            if id in industry_df[col].values:
                stock_ind.append(col)
        # for col in group_df.columns.values:
        #     if id in group_df[col].values:
        #         stock_ind.append(col)
        for col in concept_df.columns.values:
            if id in concept_df[col].values:
                stock_ind.append(col)
        alist.append((id, stock_ind))
    for ind in alist:
        if show:
            print(ind[0], f'| RS({ind[1][0]}) | CF({ind[1][1]})', '-->', end=' ')
        for i in ind[1][2:]:
            if show:
                print(i, end=' ')
            industry_category_df.append([ind[0], i])
        print('')
    industry_category_df = pd.DataFrame(industry_category_df,columns=['ID', 'category'])
    industry_category_df['count'] = 1
    industry_category_df = industry_category_df.groupby('category').sum()
    count_df = industry_category_df.sort_values(by='count', ascending=False)
    display.display(count_df.head(15))
    # ID轉成tradingview的格式
    text = ''
    for id in list(map(lambda x:x[0], alist)):
        id = str(id)
        if allstock_info.loc[id, '市場別'] == '上櫃':
            text += f'TPEX:{id}\n'
        elif allstock_info.loc[id, '市場別'] == '上市':
            text += f'TWSE:{id}\n'
        # create and save to txt
    if save_to_txt == True:
        with open(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/tradingview觀察清單/6{industry_name}.txt', 'w', encoding='utf-8') as f:
            f.write(text)
    print(f'{industry_name}.txt saved')
    delay(1)
    

ES top 35.71% : (500, 38) | CF top 57.14% : (800, 38)
CF top 57.14% & ES top 35.71% : (425, 38)
['連接器'] : (30,)
['連接器'] : (30,)
['連接器'] after fillter : (425, 38)
3217 | RS(82.7) | CF(66.0) --> 電子零件元件 連接器 TypeC apple 
3338 | RS(68.4) | CF(36.8) --> 電子零件元件 連接器 挖礦技術 PS5 散熱模組 行動支付 第三方支付 
3526 | RS(84.6) | CF(70.7) --> 電子零件元件 連接器 電動車 TypeC 寧德時代 
3533 | RS(94.2) | CF(38.2) --> 電子零件元件 連接器 TypeC 美元升值 AMD Chatgpt 
3605 | RS(71.1) | CF(39.1) --> 電子零件元件 連接器 TypeC 車用電子 自動駕駛 
3653 | RS(90.7) | CF(39.6) --> 電子零件元件 封測服務與材料 連接器 PS5 散熱模組 功率半導體 AMD Chatgpt 
6126 | RS(80.3) | CF(83.6) --> 電子零件元件 連接器 
6197 | RS(98.7) | CF(49.9) --> 電子零件元件 連接器 TypeC CES 
6290 | RS(86.3) | CF(88.1) --> 電子零件元件 連接器 apple 智慧音箱 車聯網 FANG 
6442 | RS(97.6) | CF(52.2) --> 電子零件元件 光通訊 連接器 
6715 | RS(83.5) | CF(54.9) --> 電子零件元件 傳輸介面 連接器 TypeC 
8103 | RS(65.0) | CF(57.1) --> 電子零件元件 連接器 小米 apple 虛擬貨幣 
8147 | RS(95.7) | CF(96.6) --> 電子零件元件 連接器 


,count
category,
電子零件元件,13
連接器,13
TypeC,6
apple,3
散熱模組,2
AMD,2
PS5,2
Chatgpt,2
傳輸介面,1


['連接器'].txt saved


In [26]:
# 看股票產業
individual = '1616 3167 2439'.split(' ')
buy_list = BUY_LIST
hold_list = HOLD_LIST
watch_list = WATCH_LIST
t5_list = T5_LIST
t5_2_list = T5_2_LIST
t6_list = T6_LIST
t11_list = T11_LIST
week_choose_list = WEEK_CHOOSE_LIST
day_choose_list = DAY_CHOOSE_LIST
all_ID = list(set(t5_list + t5_2_list + t6_list + t11_list))
save_to_txt = True
# 金流前20%
sort_stocks = rschoose_df.sort_values(by='busness volume(億)', ascending=False).iloc[:2000]
# ES250前20%
sort_stocks_ES = rschoose_df.sort_values(by='ES250rate', ascending=False).iloc[:2000]
# 合併共有股票
# sort_stocks = sort_stocks.loc[sort_stocks['ID'].isin(sort_stocks_ES['ID'])]
# 特定產業
industry_name = ['手機零組件']
industry_stock = all_df2.loc[all_df2['industry'].isin(industry_name)]['ID'].values
sort_stocks = sort_stocks.loc[sort_stocks['ID'].astype(str).isin(industry_stock)]['ID'].astype(str).values

# volume_sort_stocks = volume_sort_stocks.loc[volume_sort_stocks['產業別'].isin(['設備儀器廠商', '光通訊', '印刷電路板相關'])]['ID'].astype(str).values
# print('volume_sort_stocks : ', volume_sort_stocks[0], type(volume_sort_stocks[0]))
# print('all_ID : ', (all_ID))
##########################
to_watch = sort_stocks
##########################
alist = []
industry_category_df = []
show = True

for id in to_watch:
    stock_ind = []
    for col in industry_df.columns.values:
        if id in industry_df[col].values:
            stock_ind.append(col)
    for col in group_df.columns.values:
        if id in group_df[col].values:
            stock_ind.append(col)
    for col in concept_df.columns.values:
        if id in concept_df[col].values:
            stock_ind.append(col)
    alist.append((id,stock_ind))

for ind in alist:
    if show:
        print(ind[0], ':', end=' ')
    for i in ind[1]:
        if show:
            print(i, end=' ')
        industry_category_df.append([ind[0], i])
    print('')
industry_category_df = pd.DataFrame(industry_category_df,columns=['ID', 'category'])
industry_category_df['count'] = 1
industry_category_df = industry_category_df.groupby('category').sum()
count_df = industry_category_df.sort_values(by='count', ascending=False)
display.display(count_df)
## 是否要存成txt
def get_tradingview_format():
    import requests
    import pandas as pd
    import numpy as np
    import warnings
    import datetime
    warnings.filterwarnings('ignore')
    # ============上市股票df============
    url = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=1&issuetype=1&industry_code=&Page=1&chklike=Y"
    response = requests.get(url)
    listed = pd.read_html(response.text)[0]
    listed.columns = listed.iloc[0,:]
    listed = listed[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
    listed = listed.iloc[1:]

    # ============上櫃股票df============
    urlTWO = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=2&issuetype=&industry_code=&Page=1&chklike=Y"
    response = requests.get(urlTWO)
    listedTWO = pd.read_html(response.text)[0]
    listedTWO.columns = listedTWO.iloc[0,:]
    listedTWO = listedTWO.loc[listedTWO['有價證券別'] == '股票']
    listedTWO = listedTWO[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]

    # ============上市股票代號+.TW============
    stock_1 = listed["有價證券代號"]
    stock_num = stock_1.apply(lambda x: str(x) + ".TW")
    stock_num.loc[len(stock_num)+1] = '0050.TW'
    stock_num.loc[len(stock_num)+1] = '^TWII'
    # print(stock_num)

    # ============上櫃股票代號+.TWO============
    stock_2 = listedTWO["有價證券代號"]
    stock_num2 = stock_2.apply(lambda x: str(x) + ".TWO")
    # print(stock_num2)

    # ============concate全部股票代號============
    stock_num = pd.concat([stock_num, stock_num2], ignore_index=True)
    # print(stock_num)
    allstock_info = pd.concat([listed, listedTWO], ignore_index=True)
    allstock_info.columns = ["ID","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]
    allstock_info.set_index('ID', inplace = True)
    # print(allstock_info)
    return allstock_info
if save_to_txt:
    allstock_info = get_tradingview_format()
    text = ''
    for id in list(map(lambda x:x[0], alist)):
        id = str(id)
        if allstock_info.loc[id, '市場別'] == '上櫃':
            text += f'TPEX:{id}\n'
        elif allstock_info.loc[id, '市場別'] == '上市':
            text += f'TWSE:{id}\n'
        # create and save to txt
    with open(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/tradingview觀察清單/{industry_name[0]}ES.txt', 'w', encoding='utf-8') as f:
        f.write(text)
    print(f'{industry_name[0]}ES.txt saved')

2454 : IC設計 手機零組件 
3034 : IC設計 手機零組件 
8016 : IC設計 手機零組件 
4961 : IC設計 手機零組件 
3545 : IC設計 手機零組件 
2374 : 數位相機 手機零組件 
3592 : IC設計 手機零組件 
3059 : 車用電子 IC設計 數位相機 消費性電子產品 手機零組件 診斷與監測用醫材 
2402 : 印刷電路板相關 手機零組件 
2351 : 手機零組件 封測服務與材料 傳產其他 


,count
category,
手機零組件,10
IC設計,7
數位相機,2
傳產其他,1
印刷電路板相關,1
封測服務與材料,1
消費性電子產品,1
診斷與監測用醫材,1
車用電子,1


手機零組件ES.txt saved


In [ ]:
number_of_stock = []
for col in industry_df.columns.values:
    n = len(industry_df.loc[industry_df[col] != '0', col].values)
    number_of_stock.append([col, n])
number_df = pd.DataFrame(number_of_stock, columns=['industry', 'number'])

In [ ]:
# 建立所有股票產業
rs_df = pd.read_excel(r'C:\Users\User\Desktop\StockInfoHub\Stock_Data_Collector\250RS選股\2023-06-16250RS選股.xlsx').astype(str)
a = rs_df['RS 20rate'].astype(int)>=80
rs20 = rs.loc[a]
all = []
for id in rs20:
    stock_ind = []
    for col in industry_df.columns.values:
        if id in industry_df[col].values:
            stock_ind.append(col)
            all.append([col, id, 1])
rs20_df = pd.DataFrame(all, columns=['industry', 'ID', 'count'])

In [ ]:
rs_industry = all_df.groupby(by='industry').sum().sort_values(by='count',ascending=False)
rs_industry['all number'] = 1
for industry in rs_industry.index.values:
    rs_industry.loc[industry, 'all number'] = number_df.loc[number_df['industry']==industry, 'number'].values
    rs_industry.loc[industry, 'percentage'] = round(100*rs_industry.loc[industry, 'count']/rs_industry.loc[industry, 'all number'],1)

In [ ]:
rs_industry.to_excel(r'C:\Users\User\Desktop\StockInfoHub\Stock_Data_Collector\產業\2023-06-3020RS.xlsx')
rs_industry
